# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,2.56,75,66,14.05,TF,1696307472
1,1,carnarvon,-24.8667,113.6333,23.64,34,0,7.93,AU,1696307473
2,2,longyearbyen,78.2186,15.6401,-3.09,59,75,9.26,SJ,1696307473
3,3,nuuk,64.1835,-51.7216,3.34,75,75,1.54,GL,1696307473
4,4,talnakh,69.4865,88.3972,1.23,97,100,6.53,RU,1696307473


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] >= 22) & (city_data_df["Humidity"] <= 80)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna(how='any')

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,carnarvon,-24.8667,113.6333,23.64,34,0,7.93,AU,1696307473
19,19,colonia,50.9333,6.9500,22.36,74,0,4.63,DE,1696307477
22,22,port douglas,-16.4833,145.4667,25.21,72,61,4.15,AU,1696307477
36,36,saipan,15.1355,145.7010,31.39,74,40,5.66,MP,1696307480
37,37,sainte-suzanne,-20.9061,55.6089,26.17,71,0,1.03,RE,1696307481
...,...,...,...,...,...,...,...,...,...,...
567,567,sur,22.5667,59.5289,33.54,41,14,4.49,OM,1696307597
569,569,bikenibeu village,1.3673,173.1241,30.98,74,40,3.09,KI,1696307598
570,570,guiuan,11.0333,125.7247,28.90,77,84,6.52,PH,1696307598
577,577,altamira,-3.2033,-52.2064,24.79,76,19,2.31,BR,1696307600


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\Seeke\AppData\Local\Temp\ipykernel_6280\1708204268.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,75,
1,carnarvon,AU,-24.8667,113.6333,34,
2,longyearbyen,SJ,78.2186,15.6401,59,
3,nuuk,GL,64.1835,-51.7216,75,
4,talnakh,RU,69.4865,88.3972,97,
...,...,...,...,...,...,...
574,gamboma,CG,-1.8764,15.8644,90,
575,oeiras do para,BR,-2.0031,-49.8544,85,
576,dul'durga,RU,50.6761,113.5931,32,
577,altamira,BR,-3.2033,-52.2064,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
limit = 50
params = {
    "categories": "accommodation.hotel",
    "limit": limit,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = city_data_df.loc[index, "Lat"]
    longitude = city_data_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
#     # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


#     # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
#     # Convert the API response to JSON format
    name_address = name_address.json()
    
#     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port-aux-francais - nearest hotel: Keravel
carnarvon - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
nuuk - nearest hotel: Nordbo
talnakh - nearest hotel: Талнах
adamstown - nearest hotel: No hotel found
bethel - nearest hotel: Hampton Inn Danbury
iqaluit - nearest hotel: Frobisher Inn
amal - nearest hotel: Åmåls statshotell
mossel bay - nearest hotel: Protea Hotel Mossel Bay
dimako - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
nadym - nearest hotel: Янтарная
joao pinheiro - nearest hotel: Lícia
klaksvik - nearest hotel: Hotel Klaksvik
kodiak - nearest hotel: Shelikof Lodge
bilibino - nearest hotel: No hotel found
isafjordur - nearest hotel: Hótel Horn
edson - nearest hotel: Best Western High Road Inn
colonia - nearest hotel: Wasserturm Hotel Cologne
sterling - nearest hotel: Residence Inn by Marriott Dulles Airport at Dulles 28 Centre
illapel - nearest hotel: Domingo Ortiz de Rosas
port douglas - nearest hotel: 

gotse delchev - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
bamboo flat - nearest hotel: Hotel Megapode Nest
labuan bajo - nearest hotel: L BAJO HOTEL KOMODO
qaqortoq - nearest hotel: Hotel Qaqortoq
kapa'a - nearest hotel: Pono Kai Resort
rongelap - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
gorzow wielkopolski - nearest hotel: Mieszko
yichun - nearest hotel: No hotel found
balkashino - nearest hotel: Горница.
tarawa - nearest hotel: Betio Lodge
bel ombre - nearest hotel: Villa La Cachette
richmond - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
zhanibek - nearest hotel: No hotel found
mount pearl - nearest hotel: Sandman Signature St. John's
twon-brass - nearest hotel: KC Hotel
alofi - nearest hotel: Matavai Resort
orcopampa - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
kamina - nearest hotel: Hôtel DG
bonthe - nearest hotel: No

callao - nearest hotel: Casa Ronald
severo-yeniseyskiy - nearest hotel: Актолик
buulobarde - nearest hotel: No hotel found
cole harbour - nearest hotel: Hearthstone Inn Boutique Hotel Dartmouth/Halifax
leninsk-kuznetsky - nearest hotel: Гостиница Заря
mutuali - nearest hotel: No hotel found
berezovyy - nearest hotel: No hotel found
tekman - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
najran - nearest hotel: No hotel found
zuni pueblo - nearest hotel: No hotel found
sikasso - nearest hotel: No hotel found
micheweni - nearest hotel: No hotel found
obuase - nearest hotel: Pemisah Hotel
marsa alam - nearest hotel: Blue House Hotel
pervomayskoye - nearest hotel: Европа
tabuk - nearest hotel: Davidson Hotel
nuku'alofa - nearest hotel: City Hotel
arica - nearest hotel: Hotel Andalucía 
isetskoye - nearest hotel: Стриж
ploneour-lanvern - nearest hotel: Hôtel de Bretagne
carutapera - nearest hotel: No hotel found
gornji milanovac - neares

ocean shores - nearest hotel: Collins Inn & Seaside Cottages
caucete - nearest hotel: No hotel found
gamboma - nearest hotel: No hotel found
oeiras do para - nearest hotel: No hotel found
dul'durga - nearest hotel: No hotel found
altamira - nearest hotel: Hotel Vale do Xingu
china - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
0,port-aux-francais,TF,-49.3500,70.2167,75,Keravel
1,carnarvon,AU,-24.8667,113.6333,34,No hotel found
2,longyearbyen,SJ,78.2186,15.6401,59,The Vault
3,nuuk,GL,64.1835,-51.7216,75,Nordbo
4,talnakh,RU,69.4865,88.3972,97,Талнах
...,...,...,...,...,...,...
574,gamboma,CG,-1.8764,15.8644,90,No hotel found
575,oeiras do para,BR,-2.0031,-49.8544,85,No hotel found
576,dul'durga,RU,50.6761,113.5931,32,No hotel found
577,altamira,BR,-3.2033,-52.2064,76,Hotel Vale do Xingu


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City",
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)